# Inicializar

In [1]:
import gurobipy as gp
from gurobipy import GRB
import os
import re
import pandas as pd

In [ ]:
# Mostrar el modelo
def displayModel(model):
  try:
      os.remove("General_Model.lp")
  except OSError:
      pass
  model.write("General_Model.lp") # Escribir el modelo a un archivo .lp
  model.display()

# Modelo base

In [3]:
N = 5

def base_model(N):
    if N < 4:
        raise ValueError("El modelo híbrido requiere al menos 4 nodos (mínimo 3 paralelo, 1 serie).")

    model = gp.Model(f"General_Model_{N}_Nodos")

    # Parámetros
    I = range(3) # 0: Low, 1: Medium, 2: High
    U = range(N) # Nodos a desplegar
    W = range(2) # 0: Serie, 1: Paralelo

    L = 10
    c_values = {0: 5, 1: 12, 2: 20}

    # Variables
    x = model.addVars(U, I, W, vtype=GRB.BINARY, name="x") # Variable de decisión

    # Variables auxiliares
    N_s = model.addVar(vtype=GRB.INTEGER, name="N_s") # Número de nodos en serie
    N_p = model.addVar(vtype=GRB.INTEGER, name="N_p") # Número de nodos en paralelo
    z = model.addVar(vtype=GRB.INTEGER, name="z") # Variable auxiliar
    delta_Np = model.addVar(vtype=GRB.BINARY, name="delta_Np") # Variable binaria delta de kronecker

    """Restricciones comunes"""

    # Restricciones de unicidad para i y w
    model.addConstrs(
        (gp.quicksum(x[u, i, w] for i in I for w in W) == 1 for u in U), name="Unicidad_iw"
    )

    model.addConstr(N_s == gp.quicksum(x[u, i, 0] for i in I for u in U), name="Ns_def") # Número de nodos en serie
    model.addConstr(N_p == gp.quicksum(x[u, i, 1] for i in I for u in U), name="Np_def") # Número de nodos en paralelo
    model.addConstr(z * 2 == N_p * (N_p - 1), name="Quadratic_Exact") # Restricción cuadrática exacta

    # Restricción del delta de Kronecker: Si Np == 0 → delta_Np = 1, sino delta_Np = 0
    model.addGenConstrIndicator(delta_Np, True, N_p == 0, name="Kronecker_1")
    model.addGenConstrIndicator(delta_Np, False, N_p >= 1, name="Kronecker_0")

    # 🔴 Esta restricción solo aplica en el modelo híbrido
    model.addConstr(N_p >= 3, name="Np_Condition") # Número de nodos en paralelo mínimo

    NodesCost = gp.quicksum(c_values[i] * x[u, i, w] for i in I for u in U for w in W) # Costo de los nodos desplegados
    LinksCost = L * (N_s + z - delta_Np) # Costo de los enlaces desplegados
    # LinksCost = 0
    model.setObjective(NodesCost + LinksCost, GRB.MINIMIZE)

    #####

    displayModel(model) # Mostrar el modelo
    # model.setParam('OutputFlag', 0)  # No mostrar salida por defecto de Gurobi

    model.optimize()

    if model.status == GRB.OPTIMAL:
        variablesDecision = dict()
        for var in model.getVars():
            variablesDecision[var.varName] = var.x
        return model.objVal, variablesDecision
    else:
        raise Exception("No se encontró una solución óptima.")

Restricted license - for non-production use only - expires 2026-11-23


# Pruebas

In [ ]:
try:
    costo, variablesDecision = base_model(N)
    print(f"Objetivo: {costo}")
    print("Variables de decisión:")
    for varName, value in variablesDecision.items():
        print(f"{varName}: {value}")
except Exception as e:
    print(f"Error: {e}")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 34 columns and 63 nonzeros
Model fingerprint: 0x74a0c1e5
Model has 1 quadratic constraint
Model has 2 simple general constraints
  2 INDICATOR
Variable types: 0 continuous, 34 integer (31 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 8 rows and 32 columns
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 6 nonzeros
Presolved model has 1 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 1 continuous, 2 integer (0 binary)

Root rela

# Modelos heredados

In [9]:
# This class implements a series-only model by removing the hybrid constraint.
# It sets N_p to 0, meaning all nodes are in series.
class SerieModel(GeneralModel):
    def __init__(self, N):
        if N < 2:
            raise ValueError("El modelo solo serie requiere al menos 2 nodos.")
        super().__init__(N)
        self.model.remove(self.model.getConstrByName("Np_Condition"))  # 🚨 Remover restricción híbrida
        self.model.addConstr(self.N_p == 0, name="Solo_Serie")

In [10]:
# This class implements a parallel-only model by removing the hybrid constraint.
# It sets N_s to 0, meaning all nodes are in parallel.
class ParaleloModel(GeneralModel):
    def __init__(self, N):
        if N < 3:
            raise ValueError("El modelo solo paralelo requiere al menos 3 nodos.")
        super().__init__(N)
        self.model.addConstr(self.N_s == 0, name="Solo_Paralelo")